In [1]:
import localsolver
import sys
import math

In [2]:
def read_elem(filename):
    with open(filename) as f:
        return [str(elem) for elem in f.read().split()]

In [7]:
def read_input_sdvrp(filename):
    file_it = iter(read_elem(filename))

    nb_customers = int(next(file_it))
    capacity = int(next(file_it))

    demands = [None] * nb_customers
    for i in range(nb_customers):
        demands[i] = int(next(file_it))

    # Extracting the coordinates of the depot and the customers
    customers_x = [None] * nb_customers
    customers_y = [None] * nb_customers
    depot_x = float(next(file_it))
    depot_y = float(next(file_it))
    for i in range(nb_customers):
        customers_x[i] = float(next(file_it))
        customers_y[i] = float(next(file_it))

    distance_matrix = compute_distance_matrix(customers_x, customers_y)
    distance_depots = compute_distance_depots(depot_x, depot_y, customers_x, customers_y)
    return nb_customers, capacity, distance_matrix, distance_depots, demands

In [16]:
def main(instance_file, str_time_limit, sol_file):
    #
    # Read instance data
    #
    nb_customers, truck_capacity, dist_matrix_data, dist_depot_data, demands_data = \
        read_input_sdvrp(instance_file)
    nb_trucks = nb_customers

    with localsolver.LocalSolver() as ls:
        #
        # Declare the optimization model
        #
        model = ls.model

        # Sequence of customers visited by each truck
        customers_sequences = [model.list(nb_customers) for _ in range(nb_trucks)]

        # Quantity carried by each truck for each customer
        quantity = [None] * nb_trucks
        for k in range(nb_trucks):
            quantity[k] = [model.float(0, demands_data[i]) for i in range(nb_customers)]

        # All customers must be visited at least by one truck
        model.constraint(model.cover(customers_sequences))

        # Create LocalSolver arrays to be able to access them with "at" operators
        dist_matrix = model.array(dist_matrix_data)
        dist_depot = model.array(dist_depot_data)

        route_distances = [None] * nb_trucks
        trucks_used = [None] * nb_trucks

        for k in range(nb_trucks):
            sequence = customers_sequences[k]
            c = model.count(sequence)

            # A truck is used if it visits at least one customer
            trucks_used[k] = model.count(sequence) > 0

            # The quantity carried in each route must not exceed the truck capacity
            quantity_array = model.array(quantity[k])
            quantity_lambda = model.lambda_function(lambda j: quantity_array[j])
            route_quantity = model.sum(sequence, quantity_lambda)
            model.constraint(route_quantity <= truck_capacity)

            # Distance traveled by each truck
            dist_lambda = model.lambda_function(
                lambda i: model.at(dist_matrix, sequence[i - 1], sequence[i]))
            route_distances[k] = model.sum(model.range(1, c), dist_lambda) \
                + model.iif(
                    trucks_used[k],
                    dist_depot[sequence[0]] + dist_depot[sequence[c - 1]],
                    0)

        for i in range(nb_customers):
            # Each customer must receive at least its demand
            quantity_served = model.sum(
                quantity[k][i] * model.contains(customers_sequences[k], i)
                for k in range(nb_trucks))
            model.constraint(quantity_served >= demands_data[i])

        total_distance = model.sum(route_distances)

        # Objective: minimize the distance traveled
        model.minimize(total_distance)

        model.close()

        # Parameterize the solver
        ls.param.time_limit = int(str_time_limit)

        ls.solve()

        #
        # Write the solution in a file with the following format:
        # each line k contain the customers visited by the truck k
        #
        r_total_distance = total_distance.value
        customer_list = []
        quantity_list = []
        for k in range(nb_trucks):
            c_list = []
            q_list = []
            if trucks_used[k].value != 1:
                customer_list.append([])
                quantity_list.append([])
                continue
            for customer in customers_sequences[k].value:
                c_list.append(customer)
            for q in quantity[k]:
                if q.value > 0:
                    q_list.append(q.value)
                    
            customer_list.append(c_list)
            quantity_list.append(q_list)
            
    return customer_list, quantity_list

In [17]:
nb_customers, truck_capacity, dist_matrix_data, dist_depot_data, demands_data = read_input_sdvrp(r'./instances/S51D4.sd')

In [21]:
truck_capacity

160

In [19]:





def read_input_sdvrp(filename):
    file_it = iter(read_elem(filename))

    nb_customers = int(next(file_it))
    capacity = int(next(file_it))

    demands = [None] * nb_customers
    for i in range(nb_customers):
        demands[i] = int(next(file_it))

    # Extracting the coordinates of the depot and the customers
    customers_x = [None] * nb_customers
    customers_y = [None] * nb_customers
    depot_x = float(next(file_it))
    depot_y = float(next(file_it))
    for i in range(nb_customers):
        customers_x[i] = float(next(file_it))
        customers_y[i] = float(next(file_it))

    distance_matrix = compute_distance_matrix(customers_x, customers_y)
    distance_depots = compute_distance_depots(depot_x, depot_y, customers_x, customers_y)
    return nb_customers, capacity, distance_matrix, distance_depots, demands


# Compute the distance between two customers
def compute_distance_matrix(customers_x, customers_y):
    nb_customers = len(customers_x)
    distance_matrix = [[None for _ in range(nb_customers)] for _ in range(nb_customers)]
    for i in range(nb_customers):
        distance_matrix[i][i] = 0
        for j in range(nb_customers):
            dist = compute_dist(customers_x[i], customers_x[j], customers_y[i], customers_y[j])
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix


# Compute the distances to depot
def compute_distance_depots(depot_x, depot_y, customers_x, customers_y):
    nb_customers = len(customers_x)
    distance_depots = [None] * nb_customers
    for i in range(nb_customers):
        dist = compute_dist(depot_x, customers_x[i], depot_y, customers_y[i])
        distance_depots[i] = dist
    return distance_depots


def compute_dist(xi, xj, yi, yj):
    exact_dist = math.sqrt(math.pow(xi - xj, 2) + math.pow(yi - yj, 2))
    return int(math.floor(exact_dist + 0.5))

instance_file = r'./instances/S51D4.sd'
sol_file = r'./result.csv'
str_time_limit = "5"

customer_list, quantity_list = main(instance_file, str_time_limit, sol_file)


Push initial solution 100%
Model:  expressions = 9055, decisions = 2550, constraints = 101, objectives = 1
Param:  time limit = 5 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 51)
[  1 sec,   18666 itr]:         1646
[  2 sec,   18666 itr]:         1646
[  3 sec,   80000 itr]:         1623
[  4 sec,  116388 itr]:         1619
[  5 sec,  144872 itr]:         1618
[  5 sec,  144872 itr]:         1618
[ optimality gap     ]:      100.00%

144872 iterations performed in 5 seconds

Feasible solution: 
  obj    =         1618
  gap    =      100.00%
  bounds =            0


In [22]:
for q in quantity_list:
    if q != []:
        print(sum(q), len(q))

3354.8499793541296 50
3288.7866875100503 48
3421.2276625140494 50
3831.1415039802732 50
2885.912757970309 49
3561.6215276206176 50
3365.8769013550864 49
3432.395454808681 50
3478.9201577379126 50
3166.9908070255824 50
3271.5175958137615 49
3732.1431842884276 50
3334.469592556272 48
3575.2586951193566 50
3730.2741802432747 50
3263.704641761918 50
3578.819573163983 50
3107.1059521567977 47
3563.546465228285 50
3756.784786733484 50
3762.5833511149826 48
3038.922680057864 50
3467.867414503496 50
3212.450675941334 50
3378.1349446208446 49
3553.079400662767 49
3454.670838398981 50
3026.2422112778313 50
3672.058342846132 49
